# ¿Qué es el ***Crawling***?

Es un proceso por el cual un robot (o spider web) navega sistemáticamente a través de internet explorando webs y siguiendo links con el propósito de recoger el contenido web en otro sistema. Todos los buscadores utilizan estos bots, llamados crawlers, para indexar las diferentes webs en sus buscadores.

En definitiva, se trata de un proceso de rastreo web, donde se captura información general y estructura de la misma.

## Y... ¿para qué sirve?

El Crawling sirve para repasar todo el contenido y enlaces de una web con el fin de indexarlo todo en los buscadores correspondientes. Es decir, sirve para que cada actualización o nuevo contenido que se publique en una página acabe apareciendo en Google y demás motores.

Además de para indexar el contenido en los buscadores, sirve para crear una copia de partes de una web y poder procesar esos datos obtenidos.

![crawler](https://dc722jrlp2zu8.cloudfront.net/media/uploads/2019/06/17/p4x01.png)

### Hagamos algunas pruebas. Hora de crawlear!

Primero instalaremos una librería imprescindible:

In [ ]:
!pip install scrapy

: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive

In [ ]:
import scrapy

class CWSpider(scrapy.Spider):
    name = "cw"
    start_urls = ["https://www.creationwatches.com/products/seiko-75/"]
    #file = ficherito
    custom_settings = {
        'FEED_FORMAT': 'csv',
        'FEED_URI': 'watches.csv'
    }
  # Completemos los comentarios con las partes que queremos extraer

    def parse(self, response):
        for result in response.css("div.catalog_productBox"):
            image = result.css("div.imgBox img::attr(src)").get()
            name = result.css("a.pro_titel::text").get()
            print(name)
            price = result.css("div h3 del::text").get()
            print(price)
            print('{}, "{}", {}'.format(image, name, price).replace(',', ''))


In [ ]:
from scrapy.crawler import CrawlerProcess

# optional
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
})

process.crawl(CWSpider)
process.start()


## Arquitectura de **Scrapy**

La arquitectura de Scrapy, es más bien la arquitectura tipica de un scraper, y también de un crawler ya que se pueden hacer ambas cosas a la vez.

![Arquitectura_Scrapy](https://doc.scrapy.org/en/0.10.3/_images/scrapy_architecture.png)


Dicho esto...

# ¿Qué es el **Scrapping**?

Comparte las mismas características que el **Crawling** pero que se enfoca más en la **transformación de datos sin estructura** en la web (como el formato HTML) en **datos estructurados** que pueden ser almacenados y analizados en una base de datos, en un csv u otro formato de almacenamiento.

## Diferencias entre **Crawling** y **Scrapping**

El concepto de web crawling es más genérico, y se refiere a la extracción de los hipervínculos de una web.

Normalmente, cuando extraemos datos de una web vamos a tener que parsear esos datos o realizar un parsing para extraer las partes o la información que nos interesa.

![diferencias](http://prowebscraping.com/wp-content/uploads/2015/10/web-scraping-vs-web-crawling.png)

- Un crawler indexa, descubre y genera fuentes de datos
- Un scraper procesa datos con reglas logicas y extrae datos estructurados
- Se puede tener un crawler+scraper que funcionen a la vez

Generemos archivos a partir de un crawler:

In [ ]:
import os

# Change the current working directory to /tmp
os.chdir("/tmp")

# List the files in the /tmp directory
print(os.listdir())

ficherito = open("/tmp/watches.csv", "w+")

In [ ]:
import scrapy

class CWSpider_2(scrapy.Spider):
    name = "cw"
    start_urls = ["https://www.creationwatches.com/products/seiko-75/"]
    custom_settings = {
        'FEED_FORMAT': 'csv',
        'FEED_URI': 'watches.csv'
    }

    # Completemos los comentarios
    def parse(self, response):
        for result in response.css("div.catalog_productBox"):
            image = result.css("div.imgBox img::attr(src)").get()
            if image is None:
                name = result.css("a.pro_titel::text").get()
                price = result.css("div h3 del::text").get()
                yield {
                'image': image,
                'name': name,
                'price': price
            }

In [ ]:
from scrapy.crawler import CrawlerProcess

# optional
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
})

process.crawl(CWSpider_2)
process.start()


### ¿Por qué falla?

**Solo puede haber un CrawlerProcess en el entorno**. Como antes ya iniciamos el proceso, no podemos iniciarlo de nuevo.
Debemos **agregar todos los crawl que queramos al proceso y después iniciarlo**.
(Reiniciemos el entorno colab para poder hacerlo...)

In [ ]:
!pip install scrapy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import scrapy

class CWSpider(scrapy.Spider):
    name = "cw"
    start_urls = ["https://www.creationwatches.com/products/seiko-75/"]
    custom_settings = {
        'FEED_FORMAT': 'csv',
        'FEED_URI': 'watches.csv'
    }

    def parse(self, response):
        for result in response.css("div.catalog_productBox"):
            image = result.css("div.imgBox img::attr(src)").get()
            if image is None:
                name = result.css("a.pro_titel::text").get()
                price = result.css("div h3 del::text").get()
                yield {
                'image': image,
                'name': name,
                'price': price
            }

In [ ]:
class WatchSpider(scrapy.Spider):
    name = 'watchspider'
    start_urls = ["https://www.creationwatches.com/products/seiko-75/"]
    custom_settings = {
        'FEED_FORMAT': 'json',
        'FEED_URI': 'watch_data.json'
    }

    def parse(self, response):
        for result in response.css("div.catalog_productBox"):
            image = result.css("div.imgBox img::attr(src)").get()
            if image is None:
                name = result.css("a.pro_titel::text").get()
                price = result.css("div h3 del::text").get()
                yield {
                'image': image,
                'name': name,
                'price': price
    }



In [ ]:
from scrapy.crawler import CrawlerProcess

# optional
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
})

# Completemos los comentarios...
process.crawl(CWSpider)
process.crawl(WatchSpider)

process.start()

In [ ]:
!tail /content/watches.csv

In [ ]:
!tail /content/watch_data.json

In [ ]:
import json
import pprint

with open('/content/watch_data.json', 'r') as f:
    data = json.load(f)

pprint.pprint(data)

Existen otras maneras de llamar a Scrapy. Veamos:

- [Scrapy common practices](https://doc.scrapy.org/en/latest/topics/practices.html)
- [Scrapy shell](https://doc.scrapy.org/en/latest/topics/shell.html)

Existen otras librerías similares a Scrapy, como:
- [Beautiful Soup](https://beautiful-soup-4.readthedocs.io/en/latest/)
- [Selenium](https://pypi.org/project/selenium/)
- [PyQuery](https://pypi.org/project/pyquery/)